# This is  a research notebook to try to find correlation between hyperwave and volume 

In [1]:
!pip install --upgrade pip
!pip install quandl

Requirement already up-to-date: pip in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (18.0)
    100% |████████████████████████████████| 51kB 1.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


In [2]:
from datetime import datetime
from scipy.spatial import ConvexHull

import quandl
quandl.ApiConfig.api_key = "cEofBzyzyihN3fj62kp4"

import pandas as pd
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')
# This is to avoid warning when dividing by zero
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [3]:
# mydata =  pd.DataFrame(quandl.get("FRED/GDP",returns="numpy", collapse="weekly",qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high'] },))
# mydata['diff'] = mydata['open'] - mydata['close']


In [4]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(mydata)

In [5]:
# f = mydata
# # display(f['open'])
# f = f.set_index('date')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(weekly_data )

## This is the place where you setup the symbol and phases

In [52]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'data_source':'quandl',
    'symbol':'DDD',
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'data_source':'quandl',
    'symbol':'VRX',
}

data_Amazon = {
    'name':'Amazon',
    'data_source':'quandl',
    'data_source':'investopedia',
    'symbol':'AMZN',
}

data_Netflix= {
    'name':'Netflox',
    'data_source':'investopedia',
    'symbol':'NFLX',
}

data_Apple = {
    'name':'Apple',
    'data_source':'investopedia',
    'symbol':'AAPL',
}

data_Intuit = {
    'name':'Intuit',
    'data_source':'investopedia',
    'symbol':'INTU',
}

data_Gilead_Sciences = {
    'name':'Gilead Sciences',
    'data_source':'investopedia',
    'symbol':'GILD',
}



In [53]:
company_setup = data_Valeant
name = company_setup ['name'] 
data_source = company_setup ['data_source'] 
symbol = company_setup ['symbol']

# constant used for other computation
root_date = datetime(1800, 1, 1)

# Helper function 
This section contain helper function that are here to load and clean the raw data

In [61]:

class Investopedia_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe
    
    def _clean_data(self, df):
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('Date')])
        df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high',})
        df = df.set_index('date')
        df['date'] = df.index
        return df

    def _fetch_data(self): 
        url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(self._symbol, self._timeframe)
        df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
        df_price = df_list[0].dropna()
        return df_price
    
    def get_dataframe(self):
        raw_data = self._fetch_data()
        return self._clean_data(raw_data)

class Quandl_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-FRI').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        daily_tickers = quandl.get_table('WIKI/PRICES', \
                                         ticker = [self._symbol], \
                                         qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high', 'volume'] }, \
                                         date = { 'gte': '1900-01-01'}, \
                                         paginate=True) #, 'lte': '2016-12-31' 
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

sources = {
    "investopedia": Investopedia_Loader,
    "quandl": Quandl_Loader,
}

def get_historical_data(symbol, source, timeframe='weekly'):
    source_class = sources[source](symbol, timeframe)
    dir(source_class)
    return source_class.get_dataframe()
    

## Function that allow to calculate the weekid from a start date

In [17]:
def get_nb_weeks(row, base_date = root_date):
    return int((row["date"]-base_date).days/7)

def clean_raw_market_data(df):
#     df.loc[:,('date')] = pd.to_datetime(df.loc[:,('date')])
#     df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high',})
    df['is_price_closing_up'] = df.close > df.close.shift()
    df['weekId'] = df.apply (lambda row: get_nb_weeks (row),axis=1)
    return df.sort_values(by="date")

def get_weekId_max_price(df):
    return df.ix[ df.loc[:,('close')].argmax()]["weekId"]


In [10]:
def get_weekId_cartesian_product(df):
    df_from = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_from", "close": "close_from"})
    df_to = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_to", "close": "close_to"})
    df_cartesian = df_from.assign(foo=1).merge(df_to.assign(foo=1)).drop('foo', 1)

    return df_cartesian [(df_cartesian .weekId_from < df_cartesian .weekId_to)]    

# From here we start fetching the data
Above was only about setting up some basic function

In [66]:
# df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price_raw = get_historical_data(symbol, data_source, 'Weekly')

df_weekly_price = clean_raw_market_data(df_weekly_price_raw)

max_price_weekId = get_weekId_max_price(df_weekly_price)
df_weekly_price_until_max = df_weekly_price [(df_weekly_price.weekId <= max_price_weekId)]
weekId_close_from_to = get_weekId_cartesian_product(df_weekly_price_until_max)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



In [67]:
print(max_price_weekId)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_weekly_price_raw )

11248


,open,close,high,low,volume,date,is_price_closing_up,weekId
date,,,,,,,,
1994-04-04,5.500,7.00,7.0000,5.5000,9500.0,1994-04-04,False,10135
1994-04-11,5.750,5.75,5.7500,5.5000,1500.0,1994-04-11,False,10136
1994-04-18,5.250,5.25,5.7500,5.2500,2000.0,1994-04-18,False,10137
1994-04-25,4.750,4.75,4.7500,4.2500,5000.0,1994-04-25,False,10138
1994-05-02,5.250,5.00,5.7500,4.6900,46100.0,1994-05-02,True,10139
1994-05-09,5.250,6.25,6.5000,5.2500,49900.0,1994-05-09,True,10140
1994-05-16,6.000,5.50,6.2500,5.2500,13500.0,1994-05-16,False,10141
1994-05-23,7.250,6.75,7.2500,5.2500,54300.0,1994-05-23,True,10142
1994-05-30,6.160,7.00,7.2500,6.0000,15900.0,1994-05-30,True,10143


In [68]:
# print(max_price_weekId)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_weekly_price)
# df_weekly_price.ix[:df_weekly_price_until_max]

,open,close,high,low,volume,date,is_price_closing_up,weekId
date,,,,,,,,
1994-04-04,5.500,7.00,7.0000,5.5000,9500.0,1994-04-04,False,10135
1994-04-11,5.750,5.75,5.7500,5.5000,1500.0,1994-04-11,False,10136
1994-04-18,5.250,5.25,5.7500,5.2500,2000.0,1994-04-18,False,10137
1994-04-25,4.750,4.75,4.7500,4.2500,5000.0,1994-04-25,False,10138
1994-05-02,5.250,5.00,5.7500,4.6900,46100.0,1994-05-02,True,10139
1994-05-09,5.250,6.25,6.5000,5.2500,49900.0,1994-05-09,True,10140
1994-05-16,6.000,5.50,6.2500,5.2500,13500.0,1994-05-16,False,10141
1994-05-23,7.250,6.75,7.2500,5.2500,54300.0,1994-05-23,True,10142
1994-05-30,6.160,7.00,7.2500,6.0000,15900.0,1994-05-30,True,10143


In [69]:
def get_line_slope_and_origine(df, x1_col_name = 'x1', \
                               y1_col_name = 'y1', \
                               x2_col_name = 'x2', \
                               y2_col_name = 'y2', \
                               m_col_name = 'm', \
                               b_col_name = 'b'):
    df[m_col_name] = (df[y1_col_name] - df[y2_col_name]) / (df[x1_col_name] - df[x2_col_name])
    df[b_col_name] = df[y1_col_name] - (df[x1_col_name] * df[m_col_name])
    return df

In [70]:
hull = ConvexHull(df_weekly_price_until_max[['weekId', 'close']].dropna())


In [71]:

x = df_weekly_price_until_max['weekId'].values
y =  df_weekly_price_until_max['close'].values
trace0 = go.Scatter(
    x = x,
    y = y,
    mode = 'lines',
    name = 'lines'
)

layout = {
#     'xaxis': {
#         'range': [0, 7]
#     },
#     'yaxis': {
#         'range': [0, 2.5]
#     },
    'shapes': [{
            'type': 'line',
            'x0': x[pair[0]],
            'y0': y[pair[0]],
            'x1': x[pair[1]],
            'y1': y[pair[1]],
            'line': {
                'color': 'rgb(55, 128, 191)',
                'width': 3,
            }
        } for pair in hull.simplices
        # Line Vertical
        ]
}
data = [trace0]

# py.iplot(data, filename='line-mode')
fig = {
    'data': data,
    'layout': layout,
}

py.iplot(fig, filename='shapes-lines')

In [72]:
def get_mean_square_error(y_true, y_pred):
    y_square_diff = np.square(y_true-y_pred)
    return np.sum(y_square_diff) / len(y_true)

def nb_is_lower(y_true, y_pred):
#     print("nb_is_lower")
#     print (y_true)
#     print(y_pred.shape)
    lower_item = y_true[y_true <= y_pred]
#     print (lower_item)
    return len(lower_item )
    
def get_y(x, m, b):
    return x * m + b

In [73]:
hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
data_from_to = [{"x1": df_weekly_price_until_max['weekId'].iloc[pair[0]], \
   "y1": df_weekly_price_until_max['close'].iloc[pair[0]], \
   "x2": df_weekly_price_until_max['weekId'].iloc[pair[1]], \
   "y2": df_weekly_price_until_max['close'].iloc[pair[1]]} for pair in hull_results]
df = pd.DataFrame(data_from_to)

In [74]:
df = get_line_slope_and_origine(df)

In [75]:
def calculate_mean_square_error(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return get_mean_square_error(df['close'], y_pred)

def nb_cut_price_low(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return nb_is_lower(df['low'], y_pred)

df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
df['days'] = np.abs(df['x1'] - df['x2'])
df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df_weekly_price),axis=1)
df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df_weekly_price),axis=1)
df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
df['ratio_slope_y1'] = df['m']/df['y1']
df['ratio_slope_y2'] = df['m']/df['y2']

In [76]:
df.sort_values(['x1'])

,x1,x2,y1,y2,m,b,angle,days,mean_square_error,nb_is_lower,ratio_error_cut,ratio_slope_y1,ratio_slope_y2
0,10135,10228,7.00,84.50,0.833333,-8438.833333,39.805571,93,3.199170e+05,1253,255.320809,0.119048,0.009862
2,10135,10136,7.00,5.75,-1.250000,12675.750000,-51.340192,1,8.889981e+05,2,444499.026219,-0.178571,-0.217391
3,10136,10138,5.75,4.75,-0.500000,5073.750000,-26.565051,2,1.619424e+05,4,40485.596365,-0.086957,-0.105263
4,10138,10893,4.75,7.39,0.003497,-30.699430,0.200345,755,3.028963e+03,7,432.709047,0.000736,0.000473
1,10228,11248,84.50,257.53,0.169637,-1650.549843,9.627844,1020,2.122960e+04,1253,16.943020,0.002008,0.000659
6,10893,10899,7.39,7.51,0.020000,-210.470000,1.145763,6,3.062737e+03,62,49.398986,0.002706,0.002663
11,10899,10979,7.51,14.15,0.083000,-897.107000,4.744674,80,3.969321e+03,113,35.126733,0.011052,0.005866
10,10979,10981,14.15,14.60,0.225000,-2456.125000,12.680383,2,1.200832e+04,118,101.765425,0.015901,0.015411
5,10981,11089,14.60,45.27,0.283981,-3103.800648,15.853565,108,1.748305e+04,135,129.504068,0.019451,0.006273
9,11089,11109,45.27,55.28,0.500500,-5504.774500,26.587965,20,5.527978e+04,138,400.578110,0.011056,0.009054


In [77]:
print('Valeant : {}', 2.99/257.53)
print('netflix : {}', 7.065833/680.600)
print('amazon : {}', 6.250000/ 1578.89)

Valeant : {} 0.011610297829379104
netflix : {} 0.010381770496620629
amazon : {} 0.003958477158003407


In [78]:
# def get_line_slope(row):
#     return (row.close_from - row.close_to) / (row.weekId_from - row.weekId_to)

# weekId_close_from_to['slope'] = weekId_close_from_to.apply (lambda row: get_line_slope(row),axis=1)



# weekId_close_from_to

# # m = -0.110000
# # b = 1092.930000
# # m = 15.82
# # b = -177675.62
# def is_phase_below(m, b):
# #     m = row.m
# #     b = row.b
#     df = df_weekly_price
#     df_weekly_close = df[["weekId", "close"]]
#     df_weekly_close['ln_y'] = (df_weekly_close.weekId * m) + b
#     df_weekly_close['is_below'] = df_weekly_close.ln_y.le(df_weekly_close.close) | np.isclose(df_weekly_close['ln_y'], df_weekly_close['close'])
#     return df_weekly_close['is_below'].all()


# weekId_close_from_to['is_below'] = weekId_close_from_to.apply (lambda row: is_phase_below(row, df_weekly_price),axis=1)
# is_phase_below()

# weekId_close_from_to['r'] = weekId_close_from_to(weekId_close_from_to['m'], weekId_close_from_to['b'] )

# weekId_close_from_to
# import numpy as np
# import pandas as pd
# np.seterr(divide='ignore', invalid='ignore')



# df = df_weekly_price_until_max[:15].rename(columns={'weekId':'x', 'close': 'y'})


# data = {'x':[1,2,3,4],'y':[5,6,10,12]}
# df = pd.DataFrame(data)



# calculate m and b from y = mx = b
# df_m = (df['y'].values - df['y'].values[:, None]) / (df['x'].values - df['x'].values[:, None])
# df_b = df['y'].values - (df['x'].values * df_m)


# import itertools
# nb_partition = int((df['x'].values.shape[0] / 10)) + 1
# x_values_split = np.array_split(df['x'].values, nb_partition) 
# y_values_split = np.array_split(df['y'].values, nb_partition)              


# def get_y_values(x_value, m, b):
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
#     x_cube = np.reshape(x_values, cube_shape)
    
#     return (x_cube * m) + b 

# def get_is_line_above_close(x_values, y_values, m, b):
#     y_lenght = y_values.shape[0]
#     cube_shape = (y_lenght, 1, 1)
    
#     y_cube = np.reshape(y_values, cube_shape)
#     return get_y_values(x_values,m, b) < y_cube

    
# result = [get_y_vlowalues(x_values, df_m, df_b) for (x_values, y_values) in itertools.zip_longest(x_values_split, y_values_split )]
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
    
#     x_cube = np.reshape(x_values, cube_shape)
#     y_cube = np.reshape(y_values, cube_shape)
#     cube_y_value_calculated =  (x_cube * df_m) + df_b 

# x_values = x_values_split[0]
# y_values = y_values_split[0]              
# x_lenght = x_values.shape[0]
# cube_shape = (x_lenght, 1, 1)

# x_cube = np.reshape(x_values, cube_shape)
# y_cube = np.reshape(y_values, cube_shape)
